# The revenge of Perceptron - Learning XOR with TensorFlow
## Inspired by the GBC Book or the Deep Learning Book
### by Claude COULOMBE - TÉLUQ / UQAM - Montréal

### Introduction

The GBC book is worth the reading. It's definitely THE authoritative reference on Deep Learning but you should not be allergic to maths. 

That said, the main weakness of this masterpiece is the lack of practical programming exercices left to a companion web site. But to cover all the practical stuff, the book should have exceeded 775 pages that it already has. 

I dream of he same content in the form of a series of iPython Notebooks with all exercices and code samples using Keras, TensorFlow and Theano.

So now, I commit a modest contribution to my dream by coding the 6.1 Example: Learning XOR pp. 166 to 171, using TensorFlow. Ok, I know I should have used Theano which is mainly developed and maintained by the MILA lab at UdeM... maybe later.

[Goodfellow, Bengio, Courville, 2016] Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep learning. MIT Press. On line at: http://www.deeplearningbook.org/


## 6.1 Example: Learning XOR - GBC Book - Chapter 6 - pp. 166 to 171

### The XOR problem

In order to make the idea of a feedforward network (or multilayer perceptron) more concrete, in chapter 6 (Deep Feedforward Networks) the GBC book suggests a small example of a fully functioning feedforward network on a very simple task: learning the XOR (“exclusive or”) function. Below, the XOR Truth Table:

| $x_1$ | $x_2$ | $x_1$ XOR $x_2$ |
|:-:|:-:|:-------:|
| 0 | 0 |    0    |  
| 0 | 1 |    1    |
| 1 | 0 |    1    |
| 1 | 1 |    0    |

Nobody should get too excited, this is NOT a deep neural network example. It's quite the contrary, but it's an instructive illustration of a simple problem that requires a three layers perceptron (with a so-called "hidden" or "intermediary" layer) also known as one "hidden layer" perceptron. It also needs nonlinear functions called activation functions. 

The problem with XOR is that its outputs are not linearly separable. No unique straight line is able to separe the positive from the negative examples. 

<img src="images/XOR_not_linearly_separable.png" width=500 />

### The revenge of Perceptron

#### One hidden layer Neural Network can learn XOR

<u>That said, we will show now that a neural network with one hidden layer and the backpropagation algorithm can learn XOR.</u> 

Below the architecture of our shallow (not deep at all!) feedforward neural network which contains 5 neurons or units distributed on 3 layers: an input layer which contains two neurons (or units), a single hidden layer containing 2 neurons and the output layer with 1 neuron. 

<img src="images/feedforward_network_solving_XOR.png" width=400 />

The left part of the above figure shows a detailed representation of the neural network, neuron by neuron with all the connections (except the biases). Good for small NNs this notation can be too cumbersome for larger networks. At right, the neural network is presented by layers in a more compact notation with weights represented by matrix ($W$ and $w$).      

### TensorFlow implementation

Now it's time to move on and implement the XOR network in TensorFlow. I've been inspired by the blog post <a href="https://aimatters.wordpress.com/2016/01/16/solving-xor-with-a-neural-network-in-tensorflow/">"Solving XOR with a Neural Network in TensorFlow"</a> by Stephen OMAN and the <a href="https://github.com/StephenOman/TensorFlowExamples/blob/master/xor%20nn/xor_nn.py">code</a> in his GitHub repo. 

#### Playing with different loss functions

I also proposed different loss functions that I've commented in my code below. 

First, a naive direct implementation of the loss function as shown in the GBC book.

`n_instances = X.get_shape().as_list()[0]`<br/>
`loss = tf.reduce_sum(tf.pow(y_estimated - Y, 2))/ n_instances`

Then the classical MSE function suggested for math simplicity in the GBC book which uses the TensorFlow `tf.reduce_mean` function that should take care of numerical stability issue as I read somewhere...<br/>       

`loss = tf.reduce_mean(tf.squared_difference(y_estimated, Y))` 

For better result with binary classifier, use cross entropy with a sigmoid<br/>
`loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_estimated, labels=Y)`

In case of problem with gradient (exploding or vanishing gradient) we could alternatively perform gradient clipping using the TensorFlow function `tf.clip_by_value(t, clip_value_min, clip_value_max)`. Any value less than clip_value_min will be set to clip_value_min. Any value greater than clip_value_max will be set to clip_value_max.

`loss = tf.reduce_sum(tf.pow(tf.clip_by_value(y_estimated,1e-10,1.0) - Y,2))/(n_instances)`

In [5]:
# 6.1 Example: Learning XOR - GBC Book - Chapter 6 - pp. 166 to 171
# Some parts where inspired by the blog post
# Solving XOR with a Neural Network in TensorFlow
# by Stephen OMAN
# 
# https://github.com/StephenOman/TensorFlowExamples/blob/master/xor%20nn/xor_nn.py

# Activation RELU + sigmoid for binary classification output + MSE loss function
import tensorflow as tf
import time
import numpy as np

X = tf.placeholder(tf.float32, shape=[4,2], name = 'X')
Y = tf.placeholder(tf.float32, shape=[4,1], name = 'Y')

W = tf.Variable(tf.truncated_normal([2,2]), name = "W")
w = tf.Variable(tf.truncated_normal([2,1]), name = "w")

c = tf.Variable(tf.zeros([4,2]), name = "c")
b = tf.Variable(tf.zeros([4,1]), name = "b")

with tf.name_scope("hidden_layer") as scope:
    h = tf.nn.relu(tf.add(tf.matmul(X, W),c))

with tf.name_scope("output") as scope:
    y_estimated = tf.sigmoid(tf.add(tf.matmul(h,w),b))

with tf.name_scope("loss") as scope:
    loss = tf.reduce_mean(tf.squared_difference(y_estimated, Y)) 

# For better result with binary classifier, use cross entropy with a sigmoid
#    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_estimated, labels=Y)

# A naïve direct implementation of the loss function
#     n_instances = X.get_shape().as_list()[0]
#     loss = tf.reduce_sum(tf.pow(y_estimated - Y, 2))/ n_instances

# In case of problem with gradient (exploding or vanishing gradient)perform gradient clipping
#     loss = tf.reduce_sum(tf.pow(tf.clip_by_value(y_estimated,1e-10,1.0) - Y,2))/(n_instances)

with tf.name_scope("train") as scope:
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

INPUT_XOR = [[0,0],[0,1],[1,0],[1,1]]
OUTPUT_XOR = [[0],[1],[1],[0]]

init = tf.global_variables_initializer()
sess = tf.Session()

writer = tf.summary.FileWriter("./logs/xor_logs", sess.graph)

sess.run(init)

t_start = time.clock()
for epoch in range(100001):
    sess.run(train_step, feed_dict={X: INPUT_XOR, Y: OUTPUT_XOR})
    if epoch % 10000 == 0:
        print("_"*80)
        print('Epoch: ', epoch)
        print('   y_estimated: ')
        for element in sess.run(y_estimated, feed_dict={X: INPUT_XOR, Y: OUTPUT_XOR}):
            print('    ',element)
        print('   W: ')
        for element in sess.run(W):
            print('    ',element)
        print('   c: ')
        for element in sess.run(c):
            print('    ',element)
        print('   w: ')
        for element in sess.run(w):
            print('    ',element)
        print('   b ')
        for element in sess.run(b):
            print('    ',element)
        print('   loss: ', sess.run(loss, feed_dict={X: INPUT_XOR, Y: OUTPUT_XOR}))
t_end = time.clock()
print("_"*80)
print('Elapsed time ', t_end - t_start)

________________________________________________________________________________
Epoch:  0
   y_estimated: 
     [ 0.49961096]
     [ 0.9245432]
     [ 0.50023597]
     [ 0.91048223]
   W: 
     [-0.41441455 -0.10702395]
     [-1.52169275  1.46376789]
   c: 
     [ 0.  0.]
     [ 0.          0.00033887]
     [ 0.  0.]
     [ 0.         -0.00099035]
   w: 
     [-1.03013289]
     [ 1.71131086]
   b 
     [-0.00155615]
     [ 0.00019796]
     [ 0.00094385]
     [-0.00057854]
   loss:  [[ 0.97383487]
 [ 0.33412135]
 [ 0.47398791]
 [ 1.24861741]]
________________________________________________________________________________
Epoch:  10000
   y_estimated: 
     [ 0.02192736]
     [ 0.99700952]
     [ 0.95889795]
     [ 0.02208406]
   W: 
     [-0.41441455 -0.8323673 ]
     [-1.52169275  1.55813658]
   c: 
     [ 0.  0.]
     [ 0.          0.82005012]
     [ 0.  0.]
     [ 0.         -0.72633374]
   w: 
     [-1.03013289]
     [ 2.25646877]
   b 
     [-3.79784894]
     [ 0.44301692]
     [